In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [4]:
#Start with defenders 
valuations = pd.read_csv("...Premier_League_Player_Valuations_2019_2024.csv")
defenders = pd.read_csv("...defenders_percentile (2).csv")

FileNotFoundError: [Errno 2] No such file or directory: '...Premier_League_Player_Valuations_2019_2024.csv'

In [ ]:
#Selecting the market values of the plauers
player_values = valuations[["player_name","player_market_value_euro","Season"]]
player_values = player_values.rename(columns = {"Season":"year", "player_name":"Player"})

#Add a year column to allow for joins
new_defenders = defenders
new_defenders['year'] = new_defenders["scouting_period"].str.extract(r'(\d{4})').astype(float)

# Convert to integer and drop rows with missing years
new_defenders = new_defenders.dropna().astype({'year': 'int'}).reset_index(drop=True)

In [ ]:
#Merge the valuations and the stats
merged_df = pd.merge(new_defenders, player_values, on=["Player","year"])
merged_df = merged_df._get_numeric_data()
merged_df.head(20)

,BasedOnMinutes,Percentile_% of Aerials Won,Percentile_% of Dribblers Tackled,Percentile_Aerials Lost,Percentile_Aerials Won,Percentile_Assists,Percentile_Average Shot Distance,Percentile_Ball Recoveries,Percentile_Blocks,Percentile_Carries,...,Percentile_Touches (Mid 3rd),Percentile_Yellow Cards,Percentile_npxG + xAG,Percentile_npxG/Shot,Percentile_npxG: Non-Penalty xG,Percentile_xA: Expected Assists,Percentile_xAG: Exp. Assisted Goals,Percentile_xG: Expected Goals,year,player_market_value_euro
0,416,47.0,72.0,17.0,82.0,32.0,67.0,3.0,8.0,72.0,...,69.0,46.0,26.0,78.0,36.0,47.0,31.0,35.0,2023,900000.0
1,2235,53.0,9.0,49.0,54.0,48.0,19.0,31.0,15.0,51.0,...,51.0,77.0,77.0,16.0,14.0,79.0,84.0,14.0,2022,1200000.0
2,2726,73.0,33.0,88.0,47.0,60.0,38.0,49.0,34.0,89.0,...,90.0,66.0,62.0,49.0,56.0,63.0,64.0,56.0,2021,3000000.0
3,2727,54.0,53.0,52.0,54.0,17.0,31.0,70.0,48.0,46.0,...,57.0,38.0,67.0,70.0,74.0,50.0,61.0,74.0,2019,6500000.0
4,3170,67.0,68.0,83.0,37.0,93.0,4.0,72.0,16.0,51.0,...,62.0,77.0,87.0,38.0,38.0,87.0,91.0,38.0,2020,5000000.0
5,410,21.0,58.0,21.0,46.0,36.0,50.0,31.0,75.0,43.0,...,42.0,83.0,61.0,63.0,67.0,66.0,68.0,67.0,2022,25000000.0
6,723,66.0,61.0,80.0,36.0,64.0,66.0,39.0,58.0,64.0,...,61.0,35.0,11.0,51.0,31.0,28.0,18.0,30.0,2020,40000000.0
7,1435,27.0,99.0,51.0,28.0,20.0,75.0,19.0,78.0,56.0,...,38.0,75.0,70.0,89.0,83.0,48.0,51.0,83.0,2022,25000000.0
8,1714,24.0,94.0,58.0,19.0,35.0,72.0,88.0,34.0,40.0,...,44.0,94.0,44.0,77.0,66.0,49.0,40.0,66.0,2024,22000000.0
9,1780,54.0,93.0,51.0,51.0,63.0,30.0,43.0,76.0,43.0,...,54.0,49.0,20.0,22.0,6.0,49.0,46.0,6.0,2023,20000000.0


In [ ]:
#Prepare train, test and validation data

Y = merged_df['player_market_value_euro']
X = merged_df.drop('player_market_value_euro', axis=1)

#Split training, validaton and test data

from sklearn.model_selection import train_test_split

train_x, val_x, train_y, val_y = train_test_split(X, Y, random_state = 0)


In [ ]:
#import XGBoost
import xgboost as xgb
from sklearn.metrics import mean_squared_error

my_model = xgb.XGBRegressor(n_estimators = 500)

my_model.fit(train_x, train_y,
           early_stopping_rounds=50,
           eval_set=[(val_x, val_y)],
           verbose=False)

/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=500, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [ ]:
#Get predictions 
predictions = my_model.predict(val_x)
rmse = mean_squared_error(val_y, predictions, squared=False)
print(rmse)

16428169.761377301


In [ ]:
#Importing optuna to find ideal hyperparameters for the model 
import optuna
def objective(trial):
    params = {
        "objective": "reg:squarederror",
        "n_estimators": 1000,
        "verbosity": 0,
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
        "max_depth": trial.suggest_int("max_depth", 1, 10),
        "subsample": trial.suggest_float("subsample", 0.05, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.05, 1.0),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 20),
    }
    
    op_model = xgb.XGBRegressor(**params)
    op_model.fit(train_x, train_y, verbose=False)
    predictions = op_model.predict(val_x)
    rmse = mean_squared_error(val_y, predictions, squared=False)
    return rmse


In [ ]:
#Running the optuna trial
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=30)

[I 2025-02-17 11:30:12,023] A new study created in memory with name: no-name-302a9242-7059-47a5-a5e0-347c049318b9
[I 2025-02-17 11:30:13,039] Trial 0 finished with value: 17011544.28595318 and parameters: {'learning_rate': 0.061644538780808354, 'max_depth': 8, 'subsample': 0.17014174188517006, 'colsample_bytree': 0.46772877256337897, 'min_child_weight': 15}. Best is trial 0 with value: 17011544.28595318.
[I 2025-02-17 11:30:16,273] Trial 1 finished with value: 16101723.518422972 and parameters: {'learning_rate': 0.0068916307089942844, 'max_depth': 6, 'subsample': 0.7763193484623727, 'colsample_bytree': 0.3714881391182667, 'min_child_weight': 18}. Best is trial 1 with value: 16101723.518422972.
[I 2025-02-17 11:30:17,168] Trial 2 finished with value: 16885803.11189803 and parameters: {'learning_rate': 0.034528807822827794, 'max_depth': 2, 'subsample': 0.9796201740795304, 'colsample_bytree': 0.4224767571710788, 'min_child_weight': 18}. Best is trial 1 with value: 16101723.518422972.
[I 2

KeyboardInterrupt: 

In [ ]:
#Train best model
best_model = xgb.XGBRegressor(n_estimators = 1000, verbosity = 0, learning_rate = 0.01116452825266398, max_depth = 5, subsample = 0.6588063599139974,
            colsample_bytree = 0.9166236505068187, min_child_weight = 13)
best_model.fit(train_x, train_y, early_stopping_rounds = 50, eval_set=[(val_x, val_y)],verbose = False)

/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.9166236505068187, device=None,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.01116452825266398, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=13, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=1000, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [ ]:
#Get best predictions
predictions = best_model.predict(val_x)
rmse = mean_squared_error(val_y, predictions, squared=False)
print(rmse)

15993004.158200238
